In [1]:
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from functools import wraps
from sklearn.metrics import ndcg_score
from glob import glob
from os import path as osp

In [2]:
BASE_DIR = 'race_data/'
USE_COL = ['着順', '馬名', '性齢', '斤量', '騎手', 'タイム', '着差', '単勝', '人気', '馬体重', '調教師', 'horse_id', 'jockey_id']

In [3]:
p_race_csvs = sorted(glob(osp.join(BASE_DIR, 'horse_info_*.csv')))
p_race_csvs

['race_data\\horse_info_202408010101.csv',
 'race_data\\horse_info_202408010102.csv',
 'race_data\\horse_info_202408010103.csv',
 'race_data\\horse_info_202408010104.csv',
 'race_data\\horse_info_202408010106.csv',
 'race_data\\horse_info_202408010107.csv',
 'race_data\\horse_info_202408010108.csv',
 'race_data\\horse_info_202408010109.csv',
 'race_data\\horse_info_202408010110.csv',
 'race_data\\horse_info_202408010111.csv',
 'race_data\\horse_info_202408010112.csv',
 'race_data\\horse_info_202408010201.csv',
 'race_data\\horse_info_202408010202.csv',
 'race_data\\horse_info_202408010205.csv',
 'race_data\\horse_info_202408010206.csv',
 'race_data\\horse_info_202408010207.csv',
 'race_data\\horse_info_202408010208.csv',
 'race_data\\horse_info_202408010209.csv',
 'race_data\\horse_info_202408010210.csv',
 'race_data\\horse_info_202408010211.csv',
 'race_data\\horse_info_202408010212.csv',
 'race_data\\horse_info_202408010301.csv',
 'race_data\\horse_info_202408010302.csv',
 'race_data

In [4]:
def load_processed_df(p_csv, use_col=USE_COL):
    df = pd.read_csv(p_csv)
    df.columns = [c.replace(' ', '') for c in df.columns]
    df = df[use_col]
    df["レースID"] = int(osp.basename(p_csv).split('_')[2].split('.')[0])  # 各DataFrameにレースIDを付与
    df['性'] = df['性齢'].map(lambda x: x[0])
    df['齢'] = df['性齢'].map(lambda x: int(x[1:]))
    df['馬体重'] = df['馬体重'].map(lambda s: s if s != '計不' else None)
    df['当日体重'] = df['馬体重'].map(lambda s: int(s.split('(')[0]) if s is not None else None)
    df['増減'] = df['馬体重'].map(lambda s: int(s.split('(')[1].replace(')', '')) if s is not None else None)
    df.loc[df['着順'] == 1, '着差'] = 0
    df = df.drop(['性齢', '馬体重'], axis=1)
    df = df.loc[~df['着順'].isin(['取', '中', '除', '失'])]
    df['着順'] = (len(df) - df['着順'].astype(int))
    return df

In [5]:
# 複数のDataFrameをリストとして保持
dfs = [load_processed_df(p_race_csv) for p_race_csv in p_race_csvs]

# 単一のデータフレームに統合
merged_df = pd.concat(dfs, ignore_index=True)

merged_df

,着順,馬名,斤量,騎手,タイム,着差,単勝,人気,調教師,horse_id,jockey_id,レースID,性,齢,当日体重,増減
0,15,セレブレイトエール,57.0,坂井瑠星,1:55.4,0,2.6,1.0,[西] 大久保龍,2021104109,1163,202408010101,牡,3,522.0,-4.0
1,14,カフジテルビウム,57.0,長岡禎仁,1:55.9,3,14.7,5.0,[西] 杉山佳明,2021102150,1142,202408010101,牡,3,464.0,4.0
2,13,ルージュシュエット,55.0,鮫島克駿,1:55.9,アタマ,32.3,9.0,[西] 矢作芳人,2021105640,1157,202408010101,牝,3,442.0,-2.0
3,12,ヤマニンアラクリア,57.0,田中健,1:56.2,1.3/4,19.3,7.0,[西] 中村直也,2021100284,1114,202408010101,牡,3,480.0,10.0
4,11,アメリカンチーフ,57.0,松若風馬,1:56.2,クビ,5.0,2.0,[西] 音無秀孝,2021110123,1154,202408010101,牡,3,468.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7489,4,ロゼフレア,53.0,岩田望来,1:22.3,クビ,20.9,7.0,[西] 中村直也,2021103203,1174,202408070812,牝,3,452.0,6.0
7490,3,アサギリ,52.0,永島まな,1:22.3,ハナ,68.7,14.0,[東] 尾形和幸,2021100610,1187,202408070812,牝,3,460.0,-4.0
7491,2,エイムフォーエース,55.0,高杉吏麒,1:22.3,アタマ,21.7,8.0,[東] 武井亮,2021104167,1213,202408070812,牡,3,444.0,0.0
7492,1,クイックバイオ,54.0,佐々木大,1:22.4,3/4,8.4,5.0,[西] 須貝尚介,2021105374,1197,202408070812,牝,3,490.0,-4.0


In [6]:
# レースIDを取得
race_ids = merged_df["レースID"].unique()
train_ids, test_ids = train_test_split(race_ids, test_size=0.2, shuffle=False)
print(f'train size: {len(train_ids)}, test size: {len(test_ids)}')

# トレーニングデータとテストデータに分割
train_df = merged_df[merged_df["レースID"].isin(train_ids)]
test_df = merged_df[merged_df["レースID"].isin(test_ids)]

# グループ情報の再計算
train_group = train_df.groupby("レースID").size().tolist()
test_group = test_df.groupby("レースID").size().tolist()

# 特徴量とターゲットの抽出
features = ["斤量", "horse_id", "jockey_id", "人気", "当日体重", "増減"]
X_train = train_df[features]
X_test = test_df[features]
y_train = train_df["着順"]
y_test = test_df["着順"]

train size: 452, test size: 113


In [7]:
# LightGBM データセットの作成
train_data = lgb.Dataset(X_train, label=y_train, group=train_group)
test_data = lgb.Dataset(X_test, label=y_test, group=test_group, reference=train_data)

# モデルのハイパーパラメータ設定
params = {
    "objective": "lambdarank",
    "metric": "ndcg",
    "boosting": "gbdt",
    "num_leaves": 31,
    "learning_rate": 0.05,
    "verbose": -1
}

# モデルの学習
model = lgb.train(params, train_data, valid_sets=[test_data], num_boost_round=100,)

In [8]:
# テストデータでの予測
y_pred = model.predict(X_test)

# NDCGスコアの計算
true_relevance = [list(y_test)]  # 着順をリストに変換（符号は逆転済み）
predicted_scores = [y_pred]
ndcg = ndcg_score(true_relevance, predicted_scores)
print(f"NDCGスコア: {ndcg}")

NDCGスコア: 0.9220647726023844


In [9]:
test_df.loc[:, '予測重み'] = y_pred
race_ids = test_df.loc[:, 'レースID'].unique()
for race_id in race_ids:
    test_df.loc[test_df.loc[:, 'レースID'] == race_id, '予測'] = test_df.loc[test_df.loc[:, 'レースID'] == race_id, '予測重み'].rank(ascending=False, method='min')
    print(test_df.loc[test_df['レースID'] == race_id, ['レースID', '予測', '着順', '馬名']].sort_values('予測', ascending=True))
    print('---')

             レースID    予測  着順         馬名
5958  202408060504   1.0  12  ルージュシークエル
5960  202408060504   2.0  10  ランフォースマイル
5967  202408060504   3.0   3    ダノンカゼルタ
5957  202408060504   4.0  13  フォーキャンドルズ
5963  202408060504   5.0   7    セイフウサツキ
5965  202408060504   6.0   5       ディニテ
5964  202408060504   7.0   6       ツインギ
5959  202408060504   8.0  11      パスコード
5962  202408060504   9.0   8  マーゴットレジーナ
5961  202408060504  10.0   9    ジェットエアー
5968  202408060504  11.0   2  キッシングセーラー
5966  202408060504  12.0   4    ホリゾンブルー
5969  202408060504  13.0   1  ウインドラブハーツ
5970  202408060504  14.0   0   インナーリソース
---
             レースID    予測  着順         馬名
5973  202408060505   1.0  10      シームルグ
5974  202408060505   2.0   9    サンジャシント
5983  202408060505   3.0   0      コンタンゴ
5977  202408060505   4.0   6  ブラックソンソルト
5976  202408060505   5.0   7     アイブリンク
5972  202408060505   6.0  11    デリュージョン
5980  202408060505   7.0   3      セイノスケ
5981  202408060505   8.0   2     シエルルビー
5979  202408060505   9.0   4   キーチファ

C:\Users\nakag\AppData\Local\Temp\ipykernel_17972\3992871090.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df.loc[:, '予測重み'] = y_pred
C:\Users\nakag\AppData\Local\Temp\ipykernel_17972\3992871090.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df.loc[test_df.loc[:, 'レースID'] == race_id, '予測'] = test_df.loc[test_df.loc[:, 'レースID'] == race_id, '予測重み'].rank(ascending=False, method='min')


             レースID    予測  着順         馬名
6829  202408070304   1.0  10      インプレス
6831  202408070304   2.0   8   アンクルブラック
6833  202408070304   3.0   6   アスターディゴン
6832  202408070304   4.0   7   ゴールドブリーズ
6837  202408070304   5.0   2      スヴァルナ
6838  202408070304   6.0   1    デシマルサーガ
6830  202408070304   7.0   9  ナリノモンターニュ
6839  202408070304   8.0   0      メルテミア
6835  202408070304   9.0   4    ブリエヴェール
6834  202408070304  10.0   5   トーセンオリジン
6836  202408070304  11.0   3   ガイフウカイセイ
---
             レースID    予測  着順         馬名
6840  202408070305   1.0   9  ガンマジーティーピ
6843  202408070305   2.0   6   カネトシブレーブ
6841  202408070305   3.0   8    ユキノエミリオ
6842  202408070305   4.0   7   アジアンテーラー
6846  202408070305   5.0   3    クインズパフェ
6849  202408070305   6.0   0   ハイフォーエバー
6844  202408070305   7.0   5      レジポッセ
6848  202408070305   8.0   1   メイショウハント
6845  202408070305   9.0   4  レイリースピリット
6847  202408070305  10.0   2      ラベンサラ
---
             レースID   予測  着順         馬名
6850  202408070306  1.0   8  グランド

In [11]:
def calc_mean_rank(test_df, race_id):
    test_df = test_df.copy()
    sub_df = test_df.loc[test_df.loc[:, 'レースID'] == race_id, :].sort_values(
        '予測', ascending=True
    ).reset_index()
    return sub_df.loc[sub_df['着順'] == 1].index.tolist()[0] + 1

mean_rank = np.nanmean([calc_mean_rank(test_df, race_id) for race_id in test_df.loc[:, 'レースID'].unique()])
print(f'平均順位: {mean_rank}')

平均順位: 10.132743362831858
